<div style="background-color:#000;"><img src="pqn.png"></img></div>

We use math for mathematical operations, cvxopt for quadratic programming optimization, matplotlib to plot results, numpy for array calculations, and the cvxopt submodules blas and solvers to handle fast linear algebra and portfolio optimization.

## Imports and setup

This block loads all required Python libraries, configures reproducibility, and sets problem size for the simulation. It also turns off cvxopt's progress output.

In [ ]:
import math
import cvxopt as opt
import matplotlib.pyplot as plt
import numpy as np
from cvxopt import blas, solvers

In [ ]:
np.random.seed(89)
solvers.options["show_progress"] = False
n = 4
nobs = 1000

We've imported several key libraries to handle everything from mathematical functions to advanced portfolio optimization. Setting the random seed makes our results repeatable every time we run the script. Disabling the progress output from cvxopt keeps our notebook clean. The variables define the number of assets and number of simulated returns we'll use in the optimization later.

## Define helper functions and simulations

This block defines helper functions to generate random portfolio weights, simulate returns, and build random portfolios. It also includes a function that finds risk and return for Kelly-optimal portfolios at different leverage levels.

In [ ]:
def rand_weights(n):
    k = np.random.randn(n)
    return k / sum(k)

In [ ]:
def gen_returns(asset_count, nobs, drift=0.0):
    return np.random.randn(asset_count, nobs) + drift

In [ ]:
def random_portfolio(returns, weight_func):
    w = weight_func(returns.shape[0])
    mu = np.dot(np.mean(returns, axis=1), w)
    sigma = math.sqrt(np.dot(w, np.dot(np.cov(returns), w)))
    if sigma > 2:
        return random_portfolio(returns, weight_func)
    return sigma, mu

In [ ]:
def get_kelly_portfolios():
    ww = np.dot(np.linalg.inv(opt.matrix(S)), opt.matrix(pbar))
    rks = []
    res = []
    for i in np.arange(0.05, 20, 0.0001):
        w = ww / i
        rks.append(blas.dot(pbar, opt.matrix(w)))
        res.append(np.sqrt(blas.dot(opt.matrix(w), S * opt.matrix(w))))
    return res, rks

Here, we set up functions to help with all the main tasks in our simulation: creating random weights, simulating returns, and using those returns to evaluate random portfolios. One function also iterates through different leverage values to trace out the Kelly-optimal portfolios' risk and return profile. Keeping these actions modular lets us easily try different portfolio constructions and risk calculations later.

## Simulate returns and optimize portfolios

This block generates simulated returns for all assets, constructs random portfolios, and solves for the efficient frontier using quadratic programming with cvxopt.

In [ ]:
return_vec = gen_returns(n, nobs, drift=0.01)
stds, means = np.column_stack(
    [random_portfolio(return_vec, rand_weights) for _ in range(500)]
)

In [ ]:
k = np.array(return_vec)
S = opt.matrix(np.cov(k))
pbar = opt.matrix(np.mean(k, axis=1))

In [ ]:
G = -opt.matrix(np.eye(n))
h = opt.matrix(0.0, (n, 1))
A = opt.matrix(1.0, (1, n))
b = opt.matrix(1.0)

In [ ]:
N = 100
mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]

In [ ]:
portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)["x"] for mu in mus]

In [ ]:
returns = [blas.dot(pbar, x) for x in portfolios]
risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]

In [ ]:
res, rks = get_kelly_portfolios()

We create 1,000 simulated daily returns for each asset with a slight positive drift to reflect expected growth. We run hundreds of random portfolios to see the spread of risk and return, then prepare the data for quadratic programming using cvxopt matrices. We solve for the efficient frontier by varying a risk-aversion parameter and store all the optimal combinations of portfolio risk and return for plotting. We also calculate the Kelly-optimal portfolios to compare how aggressive leverage changes expected return and risk.

## Visualize portfolio results and Kelly curve

This block plots random portfolios, the efficient frontier, and Kelly portfolios to compare how each approach balances risk and expected return.

In [ ]:
f, ax = plt.subplots()
plt.plot(stds, means, "o", markersize=2)
ax.set_xlabel("Volitility")
ax.set_ylabel("Return")
plt.title("Optimal Portfolio with Kelly")
plt.plot(risks, returns, "y-o", markersize=2)
plt.plot(res, rks, color="lightgray", marker="o", markersize=1)
plt.plot(res, np.array(rks) * -1, color="lightgray", marker="o", markersize=1);

We present a chart with three key data sets: random portfolios (blue dots), the efficient frontier (yellow line and dots), and different leverage points for Kelly-optimal portfolios (light gray). This lets us see, at a glance, how different portfolio choices trade off between risk and return. We label the axes so it's easy to compare each method. The chart brings together everything from our simulation, showing clearly how different strategies perform according to risk and potential gain.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.